In [ ]:
import argparse
import torch
import numpy as np
import pandas as pd
from PIL import Image
import glob, os, re
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import random
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold

import  torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, confusion_matrix
from efficientnet_pytorch import EfficientNet

from clinical_data_loader import ImageDataset, t_ImageDataset, v_ImageDataset
from efficientnet_model import MyEffientnet_b1_clinical, model_training

importing Jupyter notebook from clinical_data_loader.ipynb
importing Jupyter notebook from clinical_data_loader_external_test.ipynb
importing Jupyter notebook from clinical_data_loader_external_validation.ipynb


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

random_seed = 123

torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
train_transforms = {
    'train_aug' : transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(0.5), #0.5
        transforms.RandomVerticalFlip(0.5), #0.5
        transforms.CenterCrop(240),
        transforms.ToTensor(),
    ]),
}

test_transforms={
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(240),
        transforms.ToTensor(),
    ]),
}


In [ ]:
img_root = 'img_root'
label_pth = 'label_root'
save_root = 'save_root'

In [ ]:
idx_root = 'patient_index_info_root'
idx_df = pd.read_excel(idx_root)

In [ ]:
train_idx = idx_df['train_number'].to_numpy()
val_idx = idx_df['val_number'].dropna().to_numpy()
test_idx = idx_df['test_number'].dropna().to_numpy()

In [ ]:
train_dataset = ImageDataset(img_root, train_idx, label_pth, 'train', transform = train_transforms['train_aug'])
val_dataset = ImageDataset(img_root, val_idx, label_pth, 'val', transform = test_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           num_workers=0, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                                          num_workers=0, pin_memory=True)

In [ ]:
t_img_root = 'external_test_img_path'
t_label_pth = 'external_test_label_path'
v_img_root = 'external_valid_img_path'
v_label_pth = 'external_valid_label_path'

In [ ]:
ex_val_dataset = v_ImageDataset(v_img_root, v_label_pth, 'val', transform = test_transforms['val'])
ex_val_loader = torch.utils.data.DataLoader(ex_val_dataset, batch_size=batch_size, shuffle=True,
                                               num_workers=0, pin_memory=True)
ex_test_dataset = t_ImageDataset(t_img_root, t_label_pth, 'test', transform = test_transforms['val'])
ex_test_loader = torch.utils.data.DataLoader(ex_test_dataset, batch_size=batch_size, shuffle=True,
                                               num_workers=0, pin_memory=True)

In [ ]:
test_dataset = ImageDataset(img_root, test_idx, label_pth, 'test', transform = test_transforms['val'])
f_test_dataset = torch.utils.data.ConcatDataset([test_dataset, ex_val_dataset])
test_loader = torch.utils.data.DataLoader(f_test_dataset, batch_size=batch_size, shuffle=True,
                                               num_workers=0, pin_memory=True)

In [ ]:
my_model = MyEffientnet_b1_clinical(512, 512, 256, 32, 16, 16, 16, 32)
my_model.cuda()

In [ ]:
num_epoch, batch_size = 30, 32

In [ ]:
training(num_epoch, my_model, train_loader, val_loader, ex_test_loader, save_root)